## Notes

- Goal: simulate the flow of voters through precincts
- The info generated by my simulator could be used by an election official as a crude lower bound to help determine the # of voting booths needed at a precinct
- *M/M/N* queue:
    - First *M* indicates that arrivals (voters arriving at the polls) obey a Markov process
    - Second *M* indicates that the service times (the amount of time a voter takes to complete a ballot) obey a Markov process
    - The *N* indicates that there are N servers (the number of voting booths)

## Mark's Discussion

- Each precinct has a certain number of booths
- VotGen and Precinct object
- Simulate function in the precinct
- Generate a voter, check if stopping condition is met
- gen_poisson_voter_parameters() returns (gap, voting_duration)
- Add gap to time 
- Increment # of voters in VotGen
- max voters = stopping condition
- departure = start + voting duration
- max voters as a VotGen attribute 
- Priority queue is sorted by departure time

- In the case that there are no empty booths, what is the earliest that the queue will be vacated - look at priority queue, who is going to leave first?
- When you call `get` you'll obtain the earliest queue to be vacated or soonest departure time 
- If arrival != closing time, voter is invalid 
- Run the loop over time steps
- 
`put(v1)` - returns the last element (called the front) of the list

In [2]:
import json
import random
import sys

In [3]:
def gen_poisson_voter_parameters(arrival_rate, voting_duration_rate):
    '''
    Draw gap and voting duration from exponetial distribution

    Inputs:
        arrival_rate: (float) lambda for gap
        voting_duration_rate: (float) lambda for voting duration

    Returns:
        (gap, voting duration) as a pair of floats
    '''
    return (random.expovariate(arrival_rate),
            random.expovariate(voting_duration_rate))

In [4]:
def load_precincts(precincts_filename):
    '''
    Load a precincts file.

    Inputs:
        precincts_filename: (string) name of the precincts file

    Returns:
        A tuple containing:
        - a list of precinct dictionaries
        - a seed (integer)
    '''

    try:
        config = json.load(open(precincts_filename))
    except OSError as e:
        print("{}".format(e), file=sys.stderr)
        return None

    if not isinstance(config, dict):
        raise ValueError("Configuration file syntax error: should contain a JSON object")

    # Validate seed
    if "seed" not in config or not isinstance(config["seed"], int):
        raise ValueError("Configuration file syntax error: does not contain a seed")

    # Validate precincts
    if "precincts" not in config or not isinstance(config["precincts"], list):
        raise ValueError("Configuration file syntax error: does not contain a list of precincts")

    if config["precincts"] == []:
        raise ValueError("Configuration file must contain at least one precinct")

    for p in config["precincts"]:
        if not isinstance(p, dict):
            raise ValueError("List of precincts includes an unexpected value: {}".format(p))

        if "name" not in p:
            raise ValueError("Precinct is missing 'name' field: {}".format(p))

        for f in ("hours_open", "num_booths", "num_voters", "voter_distribution"):
            if f not in p:
                raise ValueError("Precinct {} is missing '{}' field".format(p["name"], f))

        if "type" not in p["voter_distribution"]:
            raise ValueError("Precinct {} is missing 'type' field in 'voter_distribution'".format(p["name"]))

        if p["voter_distribution"]["type"] == "poisson":
            for f in ("voting_duration_rate", "arrival_rate"):
                if f not in p["voter_distribution"]:
                    raise ValueError("Precinct {} is missing '{}' field in 'voter_distribution".format(p["name"], f))
        else:
            raise ValueError("Precinct {} has an unknown voter distribution '{}".format(p["name"], p["voter_distribution"]))

    return config["precincts"], config["seed"]

In [5]:
def print_voters(voters, filename=None):
    '''
    Print the voters generated by the simulation.

    Inputs:
      voters: a list of voter objects
      filename: (string) specifies the name of a file to use,
         if included.
    '''
    if filename is None:
        file = sys.stdout
    else:
        try:
            file = open(filename, "w")
        except OSError as e:
            print(e, file=sys.stderr)
            sys.exit(1)

    print("Arrival Time   Voting Duration   Start Time    Departure Time",
          file=file)
    for v in voters:
        s = "{:10.2f}"
        none_str = "      None"
        at = s.format(v.arrival_time) if v.arrival_time else none_str
        vd = s.format(v.voting_duration) if v.voting_duration else none_str
        st = s.format(v.start_time) if v.start_time else none_str
        if v.arrival_time is None or \
           v.voting_duration is None or \
            v.start_time is None:
            dt = none_str
        else:
            dt = s.format(v.start_time + v.voting_duration)
        combined = "{}   {}       {}        {}\n"
        print(combined.format(at, vd, st, dt), file=file)

In [6]:
class Voter:
    
    # arrival_time and voting_duration are generated from Poisson
    def __init__(self, arrival_time, voting_duration, start_time, departure_time): 
        self.arrival_time = arrival_time
        self.voting_duration = voting_duration
        self.start_time = start_time 
        self.departure_time = departure_time
        # self.gap = gap
    
    def __repr__(self):
        return "<Voter object, arrival_time={}, voting_duration='{}'>".format(self.arrival_time, 
                                                                              self.voting_duration)

In [7]:
# Without next()
# When we generate a voter, we will already have the voter's arrival time and gap. 
# We don't know the start_time and departure_time until we go into the Precinct class

class VoterGenerator:
    
    def __init__(self, max_number_voters, arrival_rate, voting_duration_rate, seed):
        self.max_number_voters = max_number_voters
        self.arrival_rate = arrival_rate
        self.voting_duration_rate = voting_duration_rate
        self.num_voter = 0
        self.seed = seed
        # self.num_voter is a class attribute 
    
    def __iter__(self):
        random.seed(seed)
        while self.num_voter < self.max_number_voters:
            
            gap, voting_duration = gen_poisson_voter_parameters(self.arrival_rate, self.voting_duration_rate)
            if self.num_voter == 0:
                arrival_time = gap
            else:
                arrival_time += gap
            voter = Voter(arrival_time, voting_duration, None, None) 
            self.num_voter += 1
            yield voter # yield returns the next voter
            # the next() method is being implicitly called because it's an iterator

In [9]:
precincts, seed = load_precincts('config-single-precinct-2.json')

In [10]:
seed

1438018944

In [11]:
precincts

[{'name': 'Little Rodentia',
  'hours_open': 1,
  'num_voters': 10,
  'num_booths': 2,
  'voter_distribution': {'type': 'poisson',
   'voting_duration_rate': 0.1,
   'arrival_rate': 0.16666666666666666}}]

In [12]:
p = precincts[0]
p["voter_distribution"]["arrival_rate"] 

0.16666666666666666

In [13]:
p["voter_distribution"]["voting_duration_rate"] 

0.1

In [14]:
p["num_voters"]

10

In [15]:
vg = VoterGenerator(p["num_voters"],
                    p["voter_distribution"]["arrival_rate"],
                    p["voter_distribution"]["voting_duration_rate"],
                    seed)

In [16]:
vg

In [17]:
# for voter in vg:
#     print("{0:.2f}  {1:.2f}".format(voter.arrival_time, voter.voting_duration))

In [18]:
print_voters(list(vg))
# Convert vg into a list of voters so we can check the previous and subsequent voters

Arrival Time   Voting Duration   Start Time    Departure Time
      3.29         5.39             None              None

     12.39         5.74             None              None

     12.68        24.45             None              None

     15.78         6.08             None              None

     19.76        39.64             None              None

     20.52         5.79             None              None

     22.33        16.65             None              None

     24.23         3.07             None              None

     26.75         5.24             None              None

     29.04        15.43             None              None



In [31]:
precincts, seed = load_precincts('config-single-precinct-2.json')

In [38]:
precincts, seed

([{'name': 'Little Rodentia',
   'hours_open': 1,
   'num_voters': 10,
   'num_booths': 2,
   'voter_distribution': {'type': 'poisson',
    'voting_duration_rate': 0.1,
    'arrival_rate': 0.16666666666666666}}],
 1438018944)

In [53]:
from dataclasses import dataclass, field
from typing import Any

@dataclass(order=True)
class KeyedItem:
    key: float
    item: Any=field(compare=False)

import queue
q_booth = queue.PriorityQueue(num_booths)

In [54]:
# Procedural programming and then turn into OOP
p = precincts[0]
hours_open = p["hours_open"]
num_booths = p["num_booths"]
num_voters = p["num_voters"]
arrival_rate = p["voter_distribution"]["arrival_rate"]
voting_duration_rate = p["voter_distribution"]["voting_duration_rate"]
# ^^ We have all the info we need from the config file 
vg = VoterGenerator(num_voters, arrival_rate, voting_duration_rate, seed)
initial_list = list(vg)

In [55]:
sim_voter_list = []
for voter in initial_list:
    # check if pricint still open
    if voter.arrival_time > (hours_open * 60):
        break
        
    if not q_booth.full():
        voter.start_time = voter.arrival_time
    else: 
        last_voter = q_booth.get().item
        if voter.arrival_time < last_voter.departure_time:
            voter.start_time = last_voter.departure_time
        else:
            voter.start_time = voter.arrival_time

    voter.departure_time = voter.start_time + voter.voting_duration 
    
    v_item = KeyedItem(voter.departure_time, voter)
    q_booth.put(v_item)   # voter goes into booth
          
    sim_voter_list.append(voter)

In [56]:
print_voters(sim_voter_list)

Arrival Time   Voting Duration   Start Time    Departure Time
      3.29         5.39             3.29              8.69

     12.39         5.74            12.39             18.13

     12.68        24.45            12.68             37.13

     15.78         6.08            18.13             24.21

     19.76        39.64            24.21             63.85

     20.52         5.79            37.13             42.92

     22.33        16.65            42.92             59.57

     24.23         3.07            59.57             62.64

     26.75         5.24            62.64             67.88

     29.04        15.43            63.85             79.28



In [ ]:
# Number of booths will be a PQ
class Precinct:
    
    def __init__(self, config_filename):
        self.config_filename = config_filename

    

In [ ]:
def simulate_election_day(precincts, seed=0):
    random.seed(seed)
    while self.num_voter < self.max_number_voters:

        gap, voting_duration = gen_poisson_voter_parameters(self.arrival_rate, self.voting_duration_rate)
        if self.num_voter == 0:
            arrival_time = gap
        else:
            arrival_time += gap
        voter = Voter(arrival_time, voting_duration, None, None) 
        self.num_voter += 1
        yield voter # yield returns the next voter
        # the next() method is being implicitly called because it's an iterator